In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Delete AI model on "**Console for AITRIOS**"

This notebook explains the workflow for deleting AI model from "**Console**". <br>
<br>
Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import errno
import json
from pathlib import Path

import jsonschema
from console_access_library.client import Client

## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

## Load an instance of "**Console Access Library**" Client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [ ]:
%store -r
client_obj: Client = client_obj

## Delete AI model

In [ ]:
# Get an instance of AI model API
ai_model_obj = client_obj.get_ai_model()

# Call an API for deleting AI model
try:
    response = ai_model_obj.delete_model(model_id=json_load["model_id"])
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print("Start to delete AI model." + " \n\tmodel_id: " + json_load["model_id"])

## Check AI model for delete

In [ ]:
# Flag for deleting check
delete_flag = True

# Call an API for get AI model list
try:
    response = ai_model_obj.get_models()
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
# Check deleting
models = response.get("models", [])
for model in models:
    if model["model_id"] == json_load["model_id"]:
        delete_flag = False
        break
# View results
if delete_flag:
    print("Deleting AI model is completed." + " \n\tmodel_id: " + json_load["model_id"])
else:
    raise Exception(
        "Deleting AI model is failed." + " \n\tmodel_id: " + json_load["model_id"]
    )